## Fer2013 
It contain 48 x 48 pixel grayscale image of the face.There are seven categories
__(0-Angry, 1-Disgust,2-Fear, 3-Happy, 4-Sad, 5-Suprise, 6- Neutral)__ present in the data.The CSV file contains two columns that are emotion that contains numeric code from 0-6 and a pixel column that include a string surrounding in quotes for each image.

## Import the Libraries

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Flatten,Dense,Dropout,Activation,Conv2D,BatchNormalization,MaxPooling2D
from keras.optimizers import SGD


In [2]:
emotion_data =pd.read_csv('fer2013.csv')
emotion_data.head(5)

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


We then create different lists of storing the testing and training image pixels. After this, we check if pixel belongs to training then we append it into the traning list and traning labels.Similarly for pixel belonging to the Public test, We append it to testing lists the code 

In [3]:
X_train =[]
y_train = []
X_test = []
y_test = []

for index, row in emotion_data.iterrows():
    k = row['pixels'].split(' ')
    if row['Usage'] == 'Training':
        X_train.append(np.array(k))
        y_train.append(row['emotion'])
    elif row['Usage']== 'PublicTest':
        X_test.append(np.array(k))
        y_test.append(row['emotion'])

In [4]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

X_train = X_train.reshape(X_train.shape[0],48,48,1)
X_test = X_test.reshape(X_test.shape[0],48,48,1)

y_train= np_utils.to_categorical(y_train, num_classes=7)
y_test = np_utils.to_categorical(y_test, num_classes=7)


In [5]:
print('X_test moduls',X_train[0].shape)
print(" y_test moduls",y_test[0])

X_test moduls (48, 48, 1)
 y_test moduls [1. 0. 0. 0. 0. 0. 0.]


## Normalization 

In [6]:
X_train= X_train.astype('float32')
X_test = X_test.astype('float32')

In [7]:
X_train = X_train/255.0
X_test = X_test/255.0

In [8]:
model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = (5,5), input_shape = (48, 48, 1), padding='same'))
model.add(Conv2D(64, kernel_size=(5,5), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation('relu'))

model.add(Conv2D(filters=128, kernel_size=(5,5), padding='same'))
model.add(Conv2D(128, kernel_size=(5,5), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation('relu'))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same'))
model.add(Conv2D(256, kernel_size=(3,3), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(7, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 64)        102464    
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
activation (Activation)      (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 128)       204928    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)       4

In [10]:
from tensorflow.keras.utils import to_categorical
n_epochs= 20
batch_size= 64
lr = 0.0001


In [11]:
history = model.fit(X_train,y_train,batch_size=batch_size,epochs=n_epochs,validation_data=(X_test,y_test))

Epoch 1/10
449/449 [==============================] - 1599s 4s/step - loss: 1.7987 - accuracy: 0.3255 - val_loss: 1.5655 - val_accuracy: 0.4087
Epoch 2/10
449/449 [==============================] - 1414s 3s/step - loss: 1.3124 - accuracy: 0.4976 - val_loss: 1.3545 - val_accuracy: 0.4862
Epoch 3/10
 48/449 [==>...........................] - ETA: 20:51 - loss: 1.1520 - accuracy: 0.5629

KeyboardInterrupt: 

In [ ]:
model.save('Model.h5')